![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/NER_BERT_TOKEN_CLASSIFIER.ipynb)

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload license_keys.json to the folder that opens. Otherwise, you can look at the example outputs at the bottom of the notebook.

## 1. Colab Setup

Import license keys

In [ ]:
%%capture
import os
import json

with open(r'/content/spark_nlp_for_healthcare.json') as f:
    license_keys = json.load(f)
    
locals().update(license_keys)

for k,v in license_keys.items(): 
    %set_env $k=$v

In [ ]:
license_keys['PUBLIC_VERSION']

'3.2.3'

In [ ]:
%%capture

!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh

In [ ]:
import os 

os.environ["SPARK_NLP_LICENSE"] = SPARK_NLP_LICENSE    
os.environ["SECRET"] = SECRET
os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID     
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY

In [ ]:
%%capture
! pip install --upgrade spark-nlp==$PUBLIC_VERSION findspark
! pip install --upgrade spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET
! pip install --ignore-installed spark-nlp-display

In [ ]:
# Install java
! sudo apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
! java -version

os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['PATH'] = os.environ['JAVA_HOME'] + "/bin:" + os.environ['PATH']

openjdk version "1.8.0_292"
OpenJDK Runtime Environment (build 1.8.0_292-8u292-b10-0ubuntu1~18.04-b10)
OpenJDK 64-Bit Server VM (build 25.292-b10, mixed mode)


In [ ]:
import sparknlp

print (f"sparknlp version: {sparknlp.version()}")
import pandas as pd
import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

from sparknlp.annotator import *

from sparknlp.base import *
import pyspark.sql.functions as F

# import internal package
import sparknlp_jsl
from sparknlp_jsl.annotator import *

print (f"sparknlp-jsl version: {sparknlp.version()}")

spark = sparknlp_jsl.start(license_keys['SECRET'])

sparknlp version: 3.3.0
sparknlp-jsl version: 3.3.0


## 2. Create example inputs

In [ ]:
text_list_jsl = [
             """The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d."""]

In [ ]:
text_list_drug = [
    "The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK) channel family. Here we describe the genomic organization of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene for type II diabetes mellitus in the Pima Indian population. The gene spans approximately 7.6 kb and contains one noncoding and two coding exons separated by approximately 2.2 and approximately 2.6 kb introns, respectively. We identified14 single nucleotide polymorphisms (SNPs), including one that predicts aVal366Ala substitution, and an 8 base-pair (bp) insertion/deletion. Our expression studies revealed the presence of the transcript in various human tissues including the pancreas, and two major insulin-responsive tissues: fat and skeletal muscle. The characterization of the KCNJ9 gene should facilitate further studies on the function of the KCNJ9 protein and allow evaluation of the potential role of the locus in Type II diabetes.BACKGROUND: At present, it is one of the most important issues for the treatment of breast cancer to develop the standard therapy for patients previously treated with anthracyclines and taxanes. With the objective of determining the usefulness of vinorelbine monotherapy in patients with advanced or recurrent breast cancer after standard therapy, we evaluated the efficacy and safety of vinorelbine in patients previously treated with anthracyclines and taxanes."]

In [ ]:
text_list_deid = [
    """HISTORY OF PRESENT ILLNESS: Mr. Smith is a 60-year-old white male veteran with multiple comorbidities, who has a history of bladder cancer diagnosed approximately two years ago by the VA Hospital. He underwent a resection there. He was to be admitted to the Day Hospital for cystectomy. He was seen in Urology Clinic and Radiology Clinic on 02/04/2003.

HOSPITAL COURSE: Mr. Smith presented to the Day Hospital in anticipation for Urology surgery. On evaluation, EKG, echocardiogram was abnormal, a Cardiology consult was obtained. A cardiac adenosine stress MRI was then proceeded, same was positive for inducible ischemia, mild-to-moderate inferolateral subendocardial infarction with peri-infarct ischemia. In addition, inducible ischemia seen in the inferior lateral septum. Mr. Smith underwent a left heart catheterization, which revealed two vessel coronary artery disease. The RCA, proximal was 95% stenosed and the distal 80% stenosed. The mid LAD was 85% stenosed and the distal LAD was 85% stenosed. There was four Multi-Link Vision bare metal stents placed to decrease all four lesions to 0%. Following intervention, Mr. Smith was admitted to 7 Ardmore Tower under Cardiology Service under the direction of Dr. Hart. Mr. Smith had a noncomplicated post-intervention hospital course. He was stable for discharge home on 02/07/2003 with instructions to take Plavix daily for one month and Urology is aware of the same."""]

## 3. Select the NER model, construct the pipeline and visualize the results.

Select the NER model - Models: **'bert_token_classifier_ner_jsl', 'bert_token_classifier_ner_drugs', 'bert_token_classifier_ner_deid'**

For more details: https://github.com/JohnSnowLabs/spark-nlp-models#pretrained-models---spark-nlp-for-healthcare

In [ ]:
model_list = ['bert_token_classifier_ner_jsl','bert_token_classifier_ner_drugs','bert_token_classifier_ner_deid']

In [ ]:
from sparknlp_display import NerVisualizer
for MODEL_NAME in model_list:

    documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

    sentenceDetector = SentenceDetectorDLModel.pretrained() \
          .setInputCols(["document"]) \
          .setOutputCol("sentence") 

    tokenizer = Tokenizer()\
      .setInputCols("sentence")\
      .setOutputCol("token")
      
    tokenClassifier = BertForTokenClassification.pretrained( MODEL_NAME, "en", 'clinical/models')\
      .setInputCols("sentence","token")\
      .setOutputCol("ner")\
      .setCaseSensitive(True)

    ner_converter = NerConverter()\
      .setInputCols(["sentence","token","ner"])\
      .setOutputCol("ner_chunk")

    pipeline =  Pipeline(stages=[documentAssembler,sentenceDetector, tokenizer, tokenClassifier, ner_converter])
    pipelineModel = pipeline.fit(spark.createDataFrame([['']]).toDF("text"))
    
    if MODEL_NAME == "bert_token_classifier_ner_jsl":
      df = spark.createDataFrame(pd.DataFrame({"text": text_list_jsl}))
    elif MODEL_NAME == "bert_token_classifier_ner_drugs":
      df = spark.createDataFrame(pd.DataFrame({"text": text_list_drug}))
    elif MODEL_NAME == "bert_token_classifier_ner_deid":
      df = spark.createDataFrame(pd.DataFrame({"text": text_list_deid}))
    print("<----------------- MODEL NAME:","\033[1m" + MODEL_NAME + "\033[0m"," ----------------- >")
    result = pipelineModel.transform(df)
    result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
          .select(F.expr("cols['0']").alias("chunk"),
                  F.expr("cols['1']['entity']").alias("ner_label"))\
          .show(truncate=False)

    NerVisualizer().display(
        result = result.collect()[0],
        label_col = 'ner_chunk',
        document_col = 'document'
    )

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
bert_token_classifier_ner_jsl download started this may take some time.
Approximate size to download 385.8 MB
[OK!]
<----------------- MODEL NAME: bert_token_classifier_ner_jsl  ----------------- >
+---------------------------+----------------------------+
|chunk                      |ner_label                   |
+---------------------------+----------------------------+
|30-year-old                |Age                         |
|female                     |Gender                      |
|insulin dependent          |Diabetes                    |
|diabetes                   |Diabetes                    |
|type 2                     |Diabetes                    |
|coronary artery disease    |Heart_Disease               |
|chronic renal insufficiency|Kidney_Disease              |
|peripheral vascular disease|Disease_Syndrome_Disorder   |
|diabetes                   |Diabetes         

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
bert_token_classifier_ner_drugs download started this may take some time.
Approximate size to download 385.3 MB
[OK!]
<----------------- MODEL NAME: bert_token_classifier_ner_drugs  ----------------- >
+--------------+---------+
|chunk         |ner_label|
+--------------+---------+
|potassium     |DrugChem |
|nucleotide    |DrugChem |
|anthracyclines|DrugChem |
|taxanes       |DrugChem |
|vinorelbine   |DrugChem |
|vinorelbine   |DrugChem |
|anthracyclines|DrugChem |
|taxanes       |DrugChem |
+--------------+---------+



sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
bert_token_classifier_ner_deid download started this may take some time.
Approximate size to download 385.4 MB
[OK!]
<----------------- MODEL NAME: bert_token_classifier_ner_deid  ----------------- >
+---------------+---------+
|chunk          |ner_label|
+---------------+---------+
|Smith          |PATIENT  |
|60-year-old    |AGE      |
|VA Hospital    |HOSPITAL |
|Day Hospital   |HOSPITAL |
|02/04/2003     |DATE     |
|Smith          |PATIENT  |
|Day Hospital   |HOSPITAL |
|Smith          |PATIENT  |
|Smith          |PATIENT  |
|7 Ardmore Tower|STREET   |
|Hart           |DOCTOR   |
|Smith          |PATIENT  |
|02/07/2003     |DATE     |
+---------------+---------+

